In [2]:
%gui qt

Same exercise as ... but with interaction

# Create two functions

- one to select the background
- one to select the flatfield

To do this use the ImageFileDialog class

- see doc: http://www.silx.org/doc/silx/latest/modules/gui/dialog/imagefiledialog.html
- related example: examples/fileDialog.py

In [20]:
from silx.gui.dialog.ImageFileDialog import ImageFileDialog

In [34]:
import os
def getImage():
    dialog = ImageFileDialog()
    dir_ = 'data'
    assert os.path.isdir(dir_)
    dialog.setDirectory(dir_)
    if not dialog.exec_():
        return None
    else:
        return dialog.selectedImage()

In [35]:
def getFlatfield():
    return getImage()

In [36]:
def getBackground():
    return getImage()

# Create a function to correct an image from flatfield and background

take as input an image, background and flatfield. Return the normalized image

In [37]:
def normalized(image, flatfield, background):
    return (image - background) / (flatfield - background)

# Select an image and display it raw and normalized

In [38]:
from silx.gui.plot import Plot2D
def showRawAndCorrected(raw, corrected):
    plot = Plot2D()
    plot.addImage(image=raw, replace=False, legend='raw')
    plot.addImage(image=corrected, replace=False,
                  legend='corrected', offset=(0, raw.shape[0]))
    plot.show()

In [39]:
image = getImage()
background = getBackground()
flatfield = getFlatfield()
if image and background and flatfield:
    corrected = normalized(image,
                           flatfield=flatfield,
                           background=background)
    showRawAndCorrected(raw=image, corrected=corrected)

# Add an action to apply the correction on a stack of image

Here is the sample code to plot the stack of image

In [52]:
from silx.gui.plot.StackView import StackViewMainWindow
import numpy
# TODO: load data
# synthetic data, stack of 100 images of size 200x300
mystack = numpy.fromfunction(
    lambda i, j, k: numpy.sin(i/15.) + numpy.cos(j/4.) + 2 * numpy.sin(k/6.),
    (100, 200, 300)
)


sv = StackViewMainWindow()
sv.setStack(mystack)

Here is a function to apply the corection on the stack

In [63]:
def applyCorrection(images, flatfield, background):
    for image in images:
        image = normalized(images, flatfield=flatfield, background=background)

To define an action:
    - heritate from PlotAction
    - redefine the triggered function
    - See the tutorial on how to add an action see: http://www.silx.org/doc/silx/dev/modules/gui/plot/actions/examples.html
    - you can also use the PlotAction tutorial.ipynb

In [68]:
from silx.gui.plot.actions import PlotAction 
class CorrectImageAction(PlotAction):
    """QAction applying the correction algorithm

    :param plot: :class:`.PlotWidget` instance on which to operate
    :param parent: See :class:`QAction`
    """
    def __init__(self, plot, sv, parent=None):
        PlotAction.__init__(self,
                            plot,
                            icon='shape-circle',
                            text='process correction',
                            tooltip='apply the correction algorithm',
                            triggered=self.applyAction,
                            parent=parent)

    def applyAction(self):
        flatfield = getFlatfield('flat field')
        background = getBackground('background')
        applyCorrection(sv.getStack(),
                        flatfield=flatfield,
                        background=background)

In [69]:
sv = StackViewMainWindow()
sv.setStack(mystack)

toolBar = sv.getPlot().toolBar()
myaction = CorrectImageAction(sv.getPlot(), sv)
toolBar.addAction(myaction)
sv.show()

TypeError: 'method' object is not connected